## 概要
### 深度学习在文本生成中的使用主要来源于机器翻译（Machine Translation),在机器翻译中，我们需要将源序列(中文）转译成目标序列(English)。利用神经网络建模进行机器翻译，需要以下几个步骤：
* 分词
* 词典构建
* 文本表征
* 网络构建
* 模型训练
* 模型性能评估

## 分词
### 我们以中文为准，分词，顾名思义，就是将一个序列（一个句子，一段文字）按一定规则进行切割，划分成一个个小块(词，词组）。中文常用的分词工具有 
* [jieba](https://github.com/fxsjy/jieba)
* [hanlp](https://github.com/hankcs/HanLP)
* [斯坦福分词器](https://nlp.stanford.edu/software/segmenter.shtml)
* [哈工大LTP](https://github.com/HIT-SCIR/ltp)
* [清华大学THULAC](https://github.com/thunlp/THULAC)

## 词典构建
### 在对源(目标)序列进行分词后，需要构建相应的源(目标)词典。
> 对源(目标)序列的词进行词频统计，保留其中频率最高的top n 作为词典的元素（例如 n = 50k)

## 文本表征
### 利用词典将文本序列特征化，常用的有one-hot,bag of word


### 样例
##### 有如下语料：
* *我爱自然语言处理*
* *我爱深度学习*
* *我爱北京天安门*

> Step 1 : * 分词*
   *  我 | 爱|自然|语言|处理
   *  我|爱|深度|学习
   *  我|爱|北京|天安门
     
> Step 2: *词典构建*
    
    {我， 爱， 自然， 语言，处理，深度，学习，北京，天安门}
    
> Step 3: * 文本表征*
### 共现表征，窗口size为1

  |   | 我 | 爱| 自然| 语言|处理|深度|学习|北京|天安门|
  |---|
  |我| 0 | 3|0 |0 |0 |0 |0 |0 |0 |
  |爱|3|0|1|0|0|0|0|0|0|
  |自然|0|1|0|0|0|0|0|0|0|
  |语言|0|0|1|0|0|0|0|0|0|
  |处理|0|0|0|1|0|0|0|0|0|
  |深度|0|1|0|0|0|0|1|0|0|
  |学习|0|0|0|0|0|1|0|0|0|
  |北京|0|1|0|0|0|0|0|0|1|
  |天安门|0|0|0|0|0|0|0|1|0|

### bag of word
* 我 爱 自然 语言 处理 --> [1,1,1,1,1,0,0,0,0]

* 我 爱 深度 学习 --> [1,1,0,0,0,1,1,0,0]

* 我 爱 北京 天安门 --> [1,1,0,0,0,0,0,1,1]

### Word Embedding

![图解1](https://img-blog.csdnimg.cn/20181220125304247.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3BpcGlzb3JyeQ==,size_16,color_FFFFFF,t_70)
![图解2](http://mccormickml.com/assets/word2vec/matrix_mult_w_one_hot.png)

## 模型建构
### 1. RNN Encoder-Decoder
#### Encoder
> In the Encoder-Decoder Framework, an encoder reads the input sentence, a sequence of vectors
$X = (x_1,x_2,\cdots,x_{T_x})$ into a vector **c** .The most common approach is to use an RNN such that $$h_t = f(x_t,h_{t-1})$$
and  $$ c = q(\lbrace h_1,h_2,\cdots,h_{T_x}\rbrace)$$
where $h_t \in \mathbb{R}^n $ is a hidden state at time $t$, and $c$ is a vector generated from the sequence of a hidden states.
$f$ and $q$ are some nonlinear functions. often,using an **LSTM** as $f$ and $q\left(\lbrace h_1,h_2,\cdots,h_{T_x}\rbrace\right) = h_{T_x}$

#### Decoder
> The decoder is often trained to predict the next word $y_t$ given the context vector $c$ and all the 
previously predicted words $\lbrace y_1,y_2,\cdots,y_{t-1} \rbrace$. In other words, the decoder defines a probability over
the translation **$ y$** by decomposing the joint probability into the ordered conditionals:
$$ p(y) = \prod_{t=1}^Tp\left(y_t| \lbrace y_1,y_2,\cdots,y_{t-1},c\right)$$, where $y = \left(y_1,y_2,\cdots,y_{T_y}\right)$.
With an RNN, each conditional probability is modeled as $$p(y_t| \lbrace y_1,y_2,\cdots,y_{t-1},c) = g(y_{t-1},s_t,c)$$, where
$g$ is a nonlinear,potentially multi-layered,function that ouputs the probability of $y_t$,and $s_t$ is the hidden state of the RNN.

![Encoder-Decoder 1](https://cdn-images-1.medium.com/max/1600/1*_6-EVV3RJXD5KDjdnxztzg@2x.png)
------
![Encoder-Decoder 2](https://pic4.zhimg.com/80/v2-77e8a977fc3d43bec8b05633dc52ff9f_hd.jpg)
--------
![Encoder-Decoder 3](https://pic4.zhimg.com/v2-e0fbb46d897400a384873fc100c442db_r.jpg)
----------

### 2. Attention
#### 2.1 Bahdanau
> In a new model architecture, we define each conditional probability in Eq:
$$ p(y_i|y_1,y_2,\cdots,y_{i-1},x) = g(y_{i-1},s_i,c_i)$$, where $s_i$ is an RNN hidden state
for time i, computed by $$ s_i = f(s_{i-1},y_{i-1},c_i)$$.Here the probability is conditioned on a 
distinct context vector $c_i$ for each target word $y_i$.
The context vector $c_i$ depends on a sequence of *annotations* $(h_1,h_2,\cdots,h_{T_x})$ to which
an encoder maps the input sentence. Each annotation $h_i$ contains information about the whole input
sentence with a strong focus on the parts surrounding the $i$-th word of the input sequence.
The context vector $c_i$ is, then, computed as a weighted sum of these annotation $h_i$:
$$ c_i = \sum_{j=1}^{T_x}\alpha_{ij}h_j$$. The weight $\alpha_{ij}$ of each annotation$h_j$ is computed by
$$ \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x}\exp(e_{ik})}$$, where $$e_{ij} = a(s_{i-1},h_j) = V^T_atanh(W_as_{i-1} + U_ah_j)$$ is an alignment
model which scores how well the inputs around position $j$ and the output at position $i$ match. The score is
based on The RNN hidden state $s_{i-1}$ and the $j$-th annotation $h_j$ of the input sentence.
**Let $\alpha_{ij}$ be a probability that the target word $y_i$ is aligned to,or translated from,a source word $x_j$.Then,
the $c_i$ is the expected annotation over all the annotations with probabilites $\alpha_{ij}$

![attention 1](https://cdn-images-1.medium.com/max/1200/0*Jpp6WALMjZbjUFjP.png)

#### 2.2  Luong
##### 2.2.1 Global approach
> all source words are attended
![attention 2](https://images2015.cnblogs.com/blog/670089/201610/670089-20161012111506078-902266845.png)
from $ h_t \rightarrow a_t\rightarrow c_t\rightarrow \overline{h_t} $

![attention 3]()